# Assignment 2 Questions #
#### CSCI 4831 - Sabermetrics ####
#### Instructor: Hoenigman ####
#### Due: Wednesday, Feb 6 by 3pm ####

Name: Michael Dresser

**Using the Statcast data, answer the following questions in a Jupyter Notebook. These questions all use data from the 2017 season. I recommend that you download all of the 2017 data first and store it as a csv, and then read from the csv.** 

**Submit your notebook to the Assignment 2 link on Canvas. Include all csv files that you generate with your submission.**

1. (10 pts) Which Rockies player had the most plate appearances with bases loaded in 2017?

In [36]:
import pandas as pd
april = pd.read_csv("data/dataApril2017.csv")#, index_col=1)
may = pd.read_csv("data/dataMay2017.csv")#, index_col=1)
june = pd.read_csv("data/dataJune2017.csv")#, index_col=1)
july = pd.read_csv("data/dataJuly2017.csv")#, index_col=1)
august = pd.read_csv("data/dataAug2017.csv")#, index_col=1)
september = pd.read_csv("data/dataSept2017.csv")#, index_col=1)
october = pd.read_csv("data/dataOct2017.csv")#, index_col=1)

months = [april, may, june, july, august, september, october]

for i in range(len(months)):
    df = months[i]
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.drop("index", axis=1, inplace=True)
    #df.drop("Index")
    df = df.reindex(sorted(df.columns), axis=1)
    months[i] = df

df2017 = pd.concat(months, sort=True)

/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
s = 0
for df in months:
    s += len(df)
    
print(s)
print(len(df2017))

df2017 = df2017.reset_index(drop=True)
print(df2017.columns)

712505
712505
Index(['at_bat_number', 'away_score', 'away_team', 'ax', 'ay', 'az',
       'babip_value', 'balls', 'bat_score', 'batter', 'bb_type',
       'break_angle_deprecated', 'break_length_deprecated', 'des',
       'description', 'effective_speed', 'error',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'events', 'fielder_2', 'fielder_2.1', 'fielder_3', 'fielder_4',
       'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
       'fld_score', 'game_date', 'game_pk', 'game_type', 'game_year', 'hc_x',
       'hc_y', 'hit_distance_sc', 'hit_location', 'home_score', 'home_team',
       'if_fielding_alignment', 'inning', 'inning_topbot', 'iso_value',
       'launch_angle', 'launch_speed', 'launch_speed_angle',
       'of_fielding_alignment', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up',
       'p_throws', 'pfx_x', 'pfx_z', 'pitch_name', 'pitch_number',
       'pitch_type', 'pitcher', 'pitcher.1', 'plate_x', 'plate_z',
       'player_name'

In [38]:
df_rockies_home = df2017.loc[df2017["home_team"] == "COL"]
df_rockies_away = df2017.loc[df2017["away_team"] == "COL"]

df_rockies_home_batting = df_rockies_home.loc[df2017["inning_topbot"] == "Bot"].copy()
df_rockies_home_batting["rockies_score"] = df_rockies_home_batting["home_score"]
df_rockies_home_batting["enemy_score"] = df_rockies_home_batting["away_score"]
df_rockies_away_batting = df_rockies_away.loc[df2017["inning_topbot"] == "Top"].copy()
df_rockies_away_batting["rockies_score"] = df_rockies_away_batting["away_score"]
df_rockies_away_batting["enemy_score"] = df_rockies_away_batting["home_score"]
df_rockies_batting = pd.concat([df_rockies_home_batting, df_rockies_away_batting])

In [39]:
df_rockies_batting_loaded = df_rockies_batting.loc[(df_rockies_batting["on_1b"].notna()) &
                                                   (df_rockies_batting["on_2b"].notna()) &
                                                   (df_rockies_batting["on_3b"].notna())] 

In [40]:
first_pitch = df_rockies_batting_loaded.loc[(df_rockies_batting_loaded["balls"] == 0) &
                              (df_rockies_batting_loaded["strikes"] == 0)]

first_pitch.groupby("batter").size()

batter
435622.0     9
448602.0    14
452672.0     5
453568.0     9
467827.0    16
471865.0    16
506560.0     3
506924.0     1
518934.0    11
542881.0     2
543022.0     1
547172.0     4
571448.0    12
572863.0     2
596115.0     9
606132.0     3
608566.0     2
622608.0     2
642162.0     6
656546.0     2
dtype: int64

In [41]:
from pybaseball import playerid_reverse_lookup

playerid_reverse_lookup([467827, 471865])

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,gonzalez,carlos,471865,gonzc001,gonzaca01,7287,2008.0,2018.0
1,parra,gerardo,467827,parrg001,parrage01,8553,2009.0,2018.0


2. (10 pts) Which Rockies pitcher threw the most first-pitch strikes in 2017?

In [42]:
df_rockies_home_pitching = df_rockies_home.loc[df2017["inning_topbot"] == "Top"]
df_rockies_away_pitching = df_rockies_away.loc[df2017["inning_topbot"] == "Bot"]
df_rockies_pitching = pd.concat([df_rockies_home_pitching, df_rockies_away_pitching])

In [43]:
first_pitch = df_rockies_pitching.loc[(df_rockies_pitching["balls"] == 0) &
                        (df_rockies_pitching["strikes"] == 1)]
first_pitch.groupby("pitcher").size()

pitcher
430589.0     28
445197.0     91
450212.0     50
459429.0    124
493603.0    103
518452.0     99
518813.0    124
542881.0    191
543022.0    270
543475.0     86
543734.0    163
545064.0     13
573127.0     18
592351.0    253
607536.0    301
608032.0     69
608566.0    349
622608.0    272
623184.0    150
641438.0     14
656546.0    208
dtype: int64

In [44]:
playerid_reverse_lookup([608566])

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,marquez,german,608566,marqg001,marquge01,15038,2016.0,2018.0


3. (10 pts) How many times did a Rockies hitter cause a lead change in a game in the seventh inning or later in 2017? i.e. The Rockies were losing and after the player's plate appearance, the Rockies were winning. This doesn't mean that they ultimately won the game.

In [61]:
from numpy import nan

late_inning = df_rockies_batting.loc[(df_rockies_batting["inning"] >= 7)].copy()
late_inning = late_inning.sort_values(by=["game_date", "at_bat_number", "pitch_number"]).reset_index(drop=True)
late_inning["post_rockies_score"] = nan

for i in late_inning.index:
    if i <= 0:
        continue
    
    if late_inning.iloc[i - 1]["game_date"] == late_inning.iloc[i]["game_date"]:
        late_inning.at[i - 1, "post_rockies_score"] = late_inning.at[i, "rockies_score"]
    
late_inning.loc[(late_inning["rockies_score"] < late_inning["enemy_score"]) &
                (late_inning["post_rockies_score"] > late_inning["enemy_score"])].loc[:, ["at_bat_number", "away_score", "home_score", "rockies_score", "enemy_score", "post_rockies_score"]]

,at_bat_number,away_score,home_score,rockies_score,enemy_score,post_rockies_score
809,55.0,4.0,3.0,3.0,4.0,5.0
1507,67.0,7.0,0.0,0.0,7.0,10.0
1509,67.0,7.0,0.0,0.0,7.0,10.0
1511,68.0,7.0,0.0,0.0,7.0,10.0
1513,68.0,7.0,0.0,0.0,7.0,10.0
1515,68.0,7.0,0.0,0.0,7.0,10.0
1517,68.0,7.0,0.0,0.0,7.0,10.0
1521,69.0,7.0,0.0,0.0,7.0,10.0
1523,69.0,7.0,0.0,0.0,7.0,10.0
1525,69.0,7.0,0.0,0.0,7.0,10.0


4. (70 pts) So much is made about the Rockies pitching staff and how it's difficult to pitch at Coors Field because of the altitude. In the thin air, breaking balls don't break as much as they do at sea level, which makes them easier to hit. We might wonder if pitchers change their approach to pitching when they come to Coors Field, or do Rockies pitchers pitch differently at Coors than they do at other ballparks. 

For this question, you need to evaluate the starting pitching staff for the 2017 Rockies to determine if there is a difference in how they approach pitching at home and away. 

Select the five pitchers who started the most games for the Rockies in 2017.

Select the data for each game that they started at home, and games they started against the other NL West teams (San Diego, Los Angeles, San Francisco, and Arizona).

Compare pitch selection for each pitcher at home and away. You can use the mean number of pitch types, or a more advanced statistical measure if you prefer.

Compare the movement of their pitches at home and away. Again, you can use the mean.

Choose a situation, such as first pitch of the inning, runners on base and no on out, etc, and compare their pitch selection in that situation at home and away. 

Generate the code to answer these questions and then write about 2 pages explaining your results. You explanation should include 
* An introduction to the problem
* The data that you used
* Your process, including a summary of what your code does. (You don't need to show your code here.)
* Your results. It will be good to include some tables or graphs here.
* Limitations of your analysis, other variables not considered that could affect your findings.

In [62]:
df_rockies_home_pitching = df_rockies_home.loc[df2017["inning_topbot"] == "Top"].copy()
df_rockies_away_pitching = df_rockies_away.loc[df2017["inning_topbot"] == "Bot"].copy()
df_rockies_pitching = pd.concat([df_rockies_home_pitching, df_rockies_away_pitching])

In [109]:
top_starters = list(df_rockies_pitching.loc[df_rockies_pitching["inning"] == 1].groupby("pitcher").size().nlargest(n=5).index)

home_games = df_rockies_home_pitching.loc[df_rockies_home_pitching["pitcher"].isin(top_starters)]
away_games = df_rockies_away_pitching.loc[df_rockies_away_pitching["pitcher"].isin(top_starters)]
away_games = away_games.loc[away_games["home_team"].isin(nl_west)]

In [116]:
home_games.groupby(["pitcher", "pitch_type"]).size()
home_games.groupby(["pitcher", "pitch_type"]).size()

pitcher   pitch_type
543022.0  CH             33
          CU            121
          FC            306
          FF            545
          FT            236
592351.0  CH             22
          CU             79
          FF            458
          SL            202
607536.0  CH             98
          CU             40
          FF            377
          FT            518
          SL            392
608566.0  CH             46
          CU            309
          FF            734
          FT             51
          SL             77
622608.0  CH             22
          CU             21
          FF            913
          SL            277
dtype: int64

In [113]:
def set_pitch_movement(df):
    df = df.copy()
    df["delta_x"] = (df["release_pos_x"] - df["plate_x"]).abs()
    df["delta_z"] = (df["release_pos_z"] - df["plate_z"]).abs()
    df["delta_x"] = (df["release_pos_x"] - df["plate_x"]).abs()
    df["delta_z"] = (df["release_pos_z"] - df["plate_z"]).abs()
    return df

home_games = set_pitch_movement(home_games)
away_games = set_pitch_movement(away_games)

home_movement = home_games.groupby(["pitcher", "pitch_type"]).agg({"delta_x": "mean", "delta_z": "mean"})
away_movement = away_games.groupby(["pitcher", "pitch_type"]).agg({"delta_x": "mean", "delta_z": "mean"})

In [115]:
away_movement - home_movement

delta_x   delta_z
pitcher  pitch_type                    
543022.0 CH         -0.007773 -0.312934
         CU         -0.522227 -0.047953
         FC         -0.056784 -0.287531
         FF         -0.035627  0.269747
         FT          0.099379  0.094885
592351.0 CH         -0.659426  0.158018
         CU         -0.303158 -0.147763
         FF         -0.218172  0.062593
         FT               NaN       NaN
         SL         -0.114432 -0.008886
607536.0 CH         -0.280691  0.531626
         CU          0.259319  0.281059
         FF          0.389166  0.348896
         FT         -0.042838  0.331378
         SL          0.366274 -0.018899
608566.0 CH          0.165519  0.296209
         CU         -0.175037  0.324532
         FF         -0.217333  0.151788
         FT         -0.069556  0.031913
         SL          0.109739 -0.046934
622608.0 CH         -0.760366  0.182372
         CU         -0.117870  0.031040
         FF         -0.238079  0.293785
         SL          0.056890  0.436024

In [ ]:
""